In [1]:
from constants import img_size
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, subtract
from keras.models import Model
from keras.applications.resnet50 import ResNet50
import pandas as pd

Using TensorFlow backend.


## Model

In [2]:
class Seamese_Model:
    def __init__(self, base_model, opts, kernel_opts=None, bias_opts=None):
        self.base_model = base_model
        self.opts = opts
        self.kernel_opts = kernel_opts
        self.bias_opts = bias_opts
    
    def kernel_initializer(self, shape, name=None):
        if (self.kernel_opts != None):
            values = np.random.normal(loc=self.kernel_opts['loc'], scale=self.kernel_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def bias_initializer(self, shape, name=None):
        if (self.bias_opts != None):
            values = np.random.normal(loc=self.bias_opts['loc'], scale=self.bias_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def get_model(self):
        base_model = self.base_model(
            include_top=False, 
            weights=self.opts['weights'], 
            input_shape=self.opts['input_shape'],
            pooling=self.opts['pooling'],
        )
        
        for layer in base_model.layers:
            layer.trainable = False
        
        input_1 = Input(self.opts['input_shape'])
        input_2 = Input(self.opts['input_shape'])

        feture_vec_1 = base_model(input_1)
        feture_vec_2 = base_model(input_2)
        distance = subtract([feture_vec_1, feture_vec_2])

        prediction = Dense(1, activation='sigmoid', bias_initializer=self.bias_initializer)(distance)
        model = Model(input=[input_1, input_2], output=prediction)
        return model

In [3]:
model = Seamese_Model(
    ResNet50, 
    opts={
        'weights': 'imagenet',
        'input_shape': (img_size, img_size, 3),
        'pooling': 'avg',
    },
    kernel_opts={
        'loc': 0,
        'scale': 1e-2,
    },
    bias_opts={
        'loc': 0.5,
        'scale': 1e-2,
    }
)

model = model.get_model()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
subtract_1 (Subtract)           (None, 2048)         0           resnet50[1][0]                   
          

## Supervisor

In [90]:
class Supervisor:
    # use seed only for testing
    def __init__(self, df, seed=None):
        self.dataframe = df
        self.seed = seed
    
    def get_batch(self, n):
        np.random.seed(self.seed)
        classes = np.random.choice(self.dataframe['class'], size=n)
        batch = {'pairs': [], 'y': []}
        
        for i in range(n // 2):
            selection = self.dataframe[self.dataframe['class'] == classes[i]]
            indices = np.random.choice(selection.index, 2)
            batch['pairs'].append([
                selection[selection.index == indices[0]]['filename'].values[0],
                selection[selection.index == indices[1]]['filename'].values[0],
            ])
            batch['y'].append(1)
            
        for i in range(n // 2, n):
            selection = self.dataframe[self.dataframe['class'] != classes[i]]
            indices = np.random.choice(selection.index, 2)
            batch['pairs'].append([
                selection[selection.index == indices[0]]['filename'].values[0],
                selection[selection.index == indices[1]]['filename'].values[0],
            ])
            batch['y'].append(0)
            
        return batch

In [91]:
labels = pd.read_csv('./data/labels.csv')

In [97]:
supervisor = Supervisor(labels)

In [105]:
supervisor.get_batch(5)

{'pairs': [['00087.jpg', '03872.jpg'],
  ['05814.jpg', '02801.jpg'],
  ['01224.jpg', '07585.jpg'],
  ['02891.jpg', '05834.jpg'],
  ['04468.jpg', '06870.jpg']],
 'y': [1, 1, 0, 0, 0]}

## Code